In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('../code/')

In [3]:
import numpy as np
import os
import pandas as pd
from tqdm import tqdm
from data_loading import ImageSequence
from utils import square_pad_image

In [4]:
dataset_path = '/data/datasets/imagenet2012/'

In [5]:
train_generator = ImageSequence(dataset_path, 20, target_split='train', square_ims='center_pad')

  1%|          | 7/1001 [00:00<00:15, 63.92it/s]

generating splits


100%|██████████| 1001/1001 [00:15<00:00, 64.93it/s]


In [6]:
folders = train_generator.list_valid_folders()

In [7]:
from PIL import UnidentifiedImageError
from PIL import Image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
# import warnings
# warnings.filterwarnings('error')

In [25]:
def correct_sample(sample):
    sample['status_code'] = 'er'
    sample['fname'] = None
    return sample

def curate(folder_name, file_name='/curated_results.json'):
    ims_df = pd.read_json(folder_name + file_name)
    invalids = []
    ims = []
    for i, sample in ims_df[ims_df['status_code']==200].iterrows():
        print(sample['fname'])
        try:
            if sample['fname'] is None:
                sample = correct_sample(sample)
                ims_df.iloc[i] = sample
            else:
                im = square_pad_image(sample['fname'])
                ims.append(np.asarray(im))
        except UnidentifiedImageError:
            sample = correct_sample(sample)
            ims_df.iloc[i] = sample
        except OSError:
            sample = correct_sample(sample)
            ims_df.iloc[i] = sample
        except Exception as e:
            print(e)
            raise Exception('BREAK other error')
    print(folder_name, len(ims))
    preprocess_input(np.array(ims))
    new_json_path = folder_name + '/curated_results.json'
    ims_df.to_json(new_json_path)

In [26]:
# from multiprocessing import Pool

# # parallel
# pool = Pool(10)
# list(tqdm(pool.imap(curate, folders), total=len(folders)))
# pool.close()

# # single thread
# # for folder_name in tqdm(folders):
# #     curate(folder_name)

In [33]:
# extra curate
# curate('/data/datasets/imagenet2012/n04399382', '/results.json')